### Data Science Project $-$ Alcohol sales
_by Alexandra Zaykovskaya_


**Disclaimer:** this project does not promote alcohol consumption.

### Part 1. Data description

Due to the project being held at New Year's Eve I've decided to examine the alcohol sales and consumption.

General alcohol consumption of such beverages as wine and distilled alcohol, for instance, was chosen as the object of research, since it displays high seasonality - sales skyrocket in winter and plummet in summer.

The sales of alcohol in the US was selected for the study, since it can indicate the dynamics of alcohol consumption and thus serve as a significant proxy. Additionally, the month to month percentage change of the sales was considered.

Data on cumulative alcohol sales in the US was obtained from the [[Federal Reserve Bank of St. Louis portal]](https://fred.stlouisfed.org/series/S4248SM144NCEN) .

**Now, let us proceed to the coding parts**

In [46]:
!pip install sktime --upgrade

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import sklearn
from sktime.forecasting.ets import AutoETS
from sktime.utils.plotting import plot_series

### Part 2. Data visualisation

In [2]:
data = pd.read_excel('US_alcohol_sales.xlsx')

The three ways of data visualisation that I want to present include a line graph, a scatter plot and a histogram for both absolute value of sales and its percentage change.

**1) Line graph**

In [8]:
plot = go.Figure()

graph = go.Scatter(x = data['date'], y = data['sales'], mode = 'lines')
plot.add_trace(graph)

plot.update_layout(title = 'Continuous graph of sales absolute value', xaxis_title = 'Date', yaxis_title = 'Sales volume')

In [4]:
plot = go.Figure()

graph = go.Scatter(x = data['date'], y = data['sales change round %'], mode = 'lines')
plot.add_trace(graph)

plot.update_layout(title = 'Continuous graph of sales percentage change', xaxis_title = 'Date', yaxis_title = 'Sales percentage change')

**2) Scatter plot**

In [7]:
plot = go.Figure()

graph = go.Scatter(x = data['date'], y = data['sales'], mode = 'markers')
plot.add_trace(graph)

plot.update_layout(title = 'Scatter plot of sales absolute value', xaxis_title = 'Date', yaxis_title = 'Sales volume')

In [10]:
plot = go.Figure()

graph = go.Scatter(x = data['date'], y = data['sales change round %'], mode = 'markers')
plot.add_trace(graph)

plot.update_layout(title = 'Scatter plot of sales percentage change', xaxis_title = 'Date', yaxis_title = 'Sales percentage change')

**3) Histogram**

In [28]:
plot = go.Figure()

graph = go.Histogram(x = data['sales'])
plot.add_trace(graph)

plot.update_layout(title = 'Histogram', xaxis_title = 'Sales volume', yaxis_title = 'Frequency')

In [11]:
plot = go.Figure()

graph = go.Histogram(x = data['sales change round %'])
plot.add_trace(graph)

plot.update_layout(title = 'Histogram', xaxis_title = 'Sales percentage change', yaxis_title = 'Frequency')

**Train and Test Split**

Now, let us split the data into training and test parts using the standard proportion: 80% of observations being used for model training, 20% - for testing. Moreover, data is not shuffled, since it would make the time series analysis much harder - thus, training set consists of consequently taken observations, which constitute by volume 80% of the overall dataset.

Dataset "data" contains columns not only with monthly, but also with quarterly data. For convenience, the columns with quarterly information are eliminated before the split is made.

**Dataset with absoute sales values:**

In [12]:
dataset_abs = pd.DataFrame(data, columns = ['date','sales'])
dataset_abs_train, dataset_abs_test = train_test_split(dataset_abs, test_size = 0.2, random_state = None, shuffle = False)
dataset_abs_test

,date,sales
314,2018-04-01,11505
315,2018-05-01,13663
316,2018-06-01,14085
317,2018-07-01,12186
318,2018-08-01,13719
...,...,...
388,2024-06-01,16552
389,2024-07-01,15603
390,2024-08-01,16534
391,2024-09-01,14761


**Dataset with percentage change sales values:**

In [13]:
dataset_pct = pd.DataFrame(data, columns = ['date','sales change round %'])
dataset_pct_train, dataset_pct_test = train_test_split(dataset_pct, test_size = 0.2, random_state = None, shuffle = False)
dataset_pct_test

,date,sales change round %
314,2018-04-01,-0.0560
315,2018-05-01,0.1876
316,2018-06-01,0.0309
317,2018-07-01,-0.1348
318,2018-08-01,0.1258
...,...,...
388,2024-06-01,-0.0176
389,2024-07-01,-0.0573
390,2024-08-01,0.0597
391,2024-09-01,-0.1072


### Part 3. Trend, Seasonal Part, Noise Decomposition


In [66]:
decomposition = sm.tsa.seasonal_decompose(data['sales'], model = 'additive', period = 12)

trend_estimate = decomposition.trend
seasonal_estimate = decomposition.seasonal
residual_estimate = decomposition.resid

plot_1 = go.Figure()
plot_1.add_trace(go.Scatter(x = data['date'], y = data['sales'], mode = 'lines', name = 'General Data'))
plot_1.update_layout(title = 'Line graph of sales', xaxis_title = 'Date', yaxis_title = 'Sales')

plot_2 = go.Figure()
plot_2.add_trace(go.Scatter(x = data['date'], y = trend_estimate, mode = 'lines', name = 'Trend'))
plot_2.update_layout(title = 'Trend of sales', xaxis_title = 'Date', yaxis_title = 'Sales trend')

plot_3 = go.Figure()
plot_3.add_trace(go.Scatter(x = data['date'], y = seasonal_estimate, mode = 'lines', name = 'Seasonality'))
plot_3.update_layout(title = 'Seasonality of sales', xaxis_title = 'Date', yaxis_title = 'Sales seasonality')

plot_4 = go.Figure()
plot_4.add_trace(go.Scatter(x = data['date'], y = residual_estimate, mode = 'markers', name = 'Residuals'))
plot_4.update_layout(title = 'Residuals of sales', xaxis_title = 'Date', yaxis_title = 'Sales residuals')


plot_1.show(), plot_2.show(), plot_3.show(), plot_4.show()

(None, None, None, None)

In [67]:
decomposition = sm.tsa.seasonal_decompose(data['sales'], model = 'multiplicative', period = 12)

trend_estimate = decomposition.trend
seasonal_estimate = decomposition.seasonal
residual_estimate = decomposition.resid

plot_1 = go.Figure()
plot_1.add_trace(go.Scatter(x = data['date'], y = data['sales'], mode = 'lines', name = 'General Data'))
plot_1.update_layout(title = 'Line graph of sales', xaxis_title = 'Date', yaxis_title = 'Sales')

plot_2 = go.Figure()
plot_2.add_trace(go.Scatter(x = data['date'], y = trend_estimate, mode = 'lines', name = 'Trend'))
plot_2.update_layout(title = 'Trend of sales', xaxis_title = 'Date', yaxis_title = 'Sales trend')

plot_3 = go.Figure()
plot_3.add_trace(go.Scatter(x = data['date'], y = seasonal_estimate, mode = 'lines', name = 'Seasonality'))
plot_3.update_layout(title = 'Seasonality of sales', xaxis_title = 'Date', yaxis_title = 'Sales seasonality')

plot_4 = go.Figure()
plot_4.add_trace(go.Scatter(x = data['date'], y = residual_estimate, mode = 'markers', name = 'Residuals'))
plot_4.update_layout(title = 'Residuals of sales', xaxis_title = 'Date', yaxis_title = 'Sales residuals')


plot_1.show(), plot_2.show(), plot_3.show(), plot_4.show()

(None, None, None, None)

In [63]:
decomposition = sm.tsa.seasonal_decompose(data['sales change round %'], model = 'additive', period = 12)

trend_estimate = decomposition.trend
seasonal_estimate = decomposition.seasonal
residual_estimate = decomposition.resid

plot_1 = go.Figure()
plot_1.add_trace(go.Scatter(x = data['date'], y = data['sales change round %'], mode = 'lines', name = 'General Data'))
plot_1.update_layout(title = 'Line graph of sales percentage change', xaxis_title = 'Date', yaxis_title = 'Sales percentage change')

plot_2 = go.Figure()
plot_2.add_trace(go.Scatter(x = data['date'], y = trend_estimate, mode = 'lines', name = 'Trend'))
plot_2.update_layout(title = 'Trend of sales percentage change', xaxis_title = 'Date', yaxis_title = 'Sales percentage change trend')

plot_3 = go.Figure()
plot_3.add_trace(go.Scatter(x = data['date'], y = seasonal_estimate, mode = 'lines', name = 'Seasonality'))
plot_3.update_layout(title = 'Seasonality of sales percentage change', xaxis_title = 'Date', yaxis_title = 'Sales percentage change seasonality')

plot_4 = go.Figure()
plot_4.add_trace(go.Scatter(x = data['date'], y = residual_estimate, mode = 'markers', name = 'Residuals'))
plot_4.update_layout(title = 'Residuals of sales percentage change', xaxis_title = 'Date', yaxis_title = 'Sales percentage change residuals')


plot_1.show(), plot_2.show(), plot_3.show(), plot_4.show()

(None, None, None, None)

### Part 4. Forecasting using Random Forest
In this section I make an attempt to predict alcohol sales and alcohol sales change by considering the past observations of the data. The forecasting methods used include Random Forest Clustering used on the absolute values of sales and Random Forest Regression used on the percentage change of sales. The reason for such choice is discussed below in this part.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor

**Random Forest for Absolute Values**

The criterion used for the tree clustering is Gini index.

As it is shown below, during the Surprise part, the significant regressors for this time series data are lags of the dependent variable: L(2), L(10), L(11), L(12). Thus, they are used for the Random Forest.

Duplicating the split:

In [20]:
dataset_abs = pd.DataFrame(data, columns = ['date','sales'])
dataset_abs_train, dataset_abs_test = train_test_split(dataset_abs, test_size = 0.2, random_state = None, shuffle = False)
dataset_abs_test

,date,sales
314,2018-04-01,11505
315,2018-05-01,13663
316,2018-06-01,14085
317,2018-07-01,12186
318,2018-08-01,13719
...,...,...
388,2024-06-01,16552
389,2024-07-01,15603
390,2024-08-01,16534
391,2024-09-01,14761


Preparing the data and performing Random Forest:

In [17]:
dataset_abs_train['y_lag2'] = dataset_abs_train['sales'].shift(2)
dataset_abs_train['y_lag10'] = dataset_abs_train['sales'].shift(10)
dataset_abs_train['y_lag11'] = dataset_abs_train['sales'].shift(11)
dataset_abs_train['y_lag12'] = dataset_abs_train['sales'].shift(12)

dataset_abs_train = dataset_abs_train.dropna()

print(dataset_abs_train.head())

         date  sales  y_lag2  y_lag10  y_lag11  y_lag12
12 1993-02-01   3261  4936.0   4564.0   4002.0   3458.0
13 1993-03-01   4160  3031.0   4221.0   4564.0   4002.0
14 1993-04-01   4377  3261.0   4529.0   4221.0   4564.0
15 1993-05-01   4307  4160.0   4466.0   4529.0   4221.0
16 1993-06-01   4696  4377.0   4137.0   4466.0   4529.0


In [18]:
dataset_abs_test['y_lag2'] = dataset_abs_test['sales'].shift(2)
dataset_abs_test['y_lag10'] = dataset_abs_test['sales'].shift(10)
dataset_abs_test['y_lag11'] = dataset_abs_test['sales'].shift(11)
dataset_abs_test['y_lag12'] = dataset_abs_test['sales'].shift(12)

dataset_abs_test = dataset_abs_test.dropna()

print(dataset_abs_test.head())

          date  sales   y_lag2  y_lag10  y_lag11  y_lag12
326 2019-04-01  12918  10766.0  14085.0  13663.0  11505.0
327 2019-05-01  14368  12184.0  12186.0  14085.0  13663.0
328 2019-06-01  14085  12918.0  13719.0  12186.0  14085.0
329 2019-07-01  13319  14368.0  11889.0  13719.0  12186.0
330 2019-08-01  14003  14085.0  13473.0  11889.0  13719.0


In [19]:
y_abs_train = dataset_abs_train['sales']
x_abs_train = pd.DataFrame(dataset_abs_train, columns = ['y_lag2','y_lag10', 'y_lag11', 'y_lag12'])
y_abs_test = dataset_abs_test['sales']
x_abs_test = pd.DataFrame(dataset_abs_test, columns = ['y_lag2','y_lag10', 'y_lag11', 'y_lag12'])

x_abs_test, y_abs_test

(      y_lag2  y_lag10  y_lag11  y_lag12
 326  10766.0  14085.0  13663.0  11505.0
 327  12184.0  12186.0  14085.0  13663.0
 328  12918.0  13719.0  12186.0  14085.0
 329  14368.0  11889.0  13719.0  12186.0
 330  14085.0  13473.0  11889.0  13719.0
 ..       ...      ...      ...      ...
 388  15058.0  16896.0  14265.0  17592.0
 389  16849.0  15593.0  16896.0  14265.0
 390  16552.0  15903.0  15593.0  16896.0
 391  15603.0  17173.0  15903.0  15593.0
 392  16534.0  17356.0  17173.0  15903.0
 
 [67 rows x 4 columns],
 326    12918
 327    14368
 328    14085
 329    13319
 330    14003
        ...  
 388    16552
 389    15603
 390    16534
 391    14761
 392    16315
 Name: sales, Length: 67, dtype: int64)

In [ ]:
random_forest = DecisionTreeClassifier(max_depth = 100, criterion = "gini")
random_forest.fit(x_abs_train, y_abs_train)
y_predictions = random_forest.predict(x_abs_test)

In [ ]:
classes = []

for i in range(len(random_forest.classes_)):
    classes.append(str(random_forest.classes_[i]))

import graphviz
dot_data = sklearn.tree.export_graphviz(random_forest, out_file=None,
                     feature_names=x_test.columns,
                      class_names = classes,
                    filled=True, rounded=True,
                      special_characters=True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
score_train = sklearn.metrics.accuracy_score(y_true = y_abs_train, y_pred = random_forest.predict(x_abs_train))
score_test = sklearn.metrics.accuracy_score(y_true = y_abs_test, y_pred = random_forest.predict(x_abs_test))

print(score_train, score_test)

It can be noticed that, although accuracy score for the train data is rather high, it is very low for the test data. The reason is that the  exact values of regressors are used, which do not match exactly the ones used in the training set, and the algorithm is making an attempt to make exact point predictions of the dependent variables (sales) as classes.

Due to this fact, using monthly percentage change of alcohol sales or regression estimation of the dependent variable values for the Random Forest algorithm could provide better estimation of the test set.

Let us make an attempt to make regression using percentage change of sales as the dependent variable and its lags (same as for absolute values) as regressors.

Duplicating the split:

In [22]:
dataset_pct = pd.DataFrame(data, columns = ['date','sales change round %'])
dataset_pct_train, dataset_pct_test = train_test_split(dataset_pct, test_size = 0.2, random_state = None, shuffle = False)
dataset_pct_train

,date,sales change round %
0,1992-02-01,-0.0003
1,1992-03-01,0.1573
2,1992-04-01,0.1404
3,1992-05-01,-0.0752
4,1992-06-01,0.0730
...,...,...
309,2017-11-01,0.0417
310,2017-12-01,0.0739
311,2018-01-01,-0.3330
312,2018-02-01,0.0872


Preparing the data and performing Random Forest:

In [23]:
dataset_pct_train['y_lag2'] = dataset_pct_train['sales change round %'].shift(2)
dataset_pct_train['y_lag10'] = dataset_pct_train['sales change round %'].shift(10)
dataset_pct_train['y_lag11'] = dataset_pct_train['sales change round %'].shift(11)
dataset_pct_train['y_lag12'] = dataset_pct_train['sales change round %'].shift(12)

dataset_pct_train = dataset_pct_train.dropna()

dataset_pct_train.head()

,date,sales change round %,y_lag2,y_lag10,y_lag11,y_lag12
12,1993-02-01,0.0759,0.1642,0.1404,0.1573,-0.0003
13,1993-03-01,0.2757,-0.3859,-0.0752,0.1404,0.1573
14,1993-04-01,0.0522,0.0759,0.0730,-0.0752,0.1404
15,1993-05-01,-0.0160,0.2757,-0.0139,0.0730,-0.0752
16,1993-06-01,0.0903,0.0522,-0.0737,-0.0139,0.0730


In [24]:
dataset_pct_test['y_lag2'] = dataset_pct_test['sales change round %'].shift(2)
dataset_pct_test['y_lag10'] = dataset_pct_test['sales change round %'].shift(10)
dataset_pct_test['y_lag11'] = dataset_pct_test['sales change round %'].shift(11)
dataset_pct_test['y_lag12'] = dataset_pct_test['sales change round %'].shift(12)

dataset_pct_test = dataset_pct_test.dropna()

dataset_pct_test.head()

,date,sales change round %,y_lag2,y_lag10,y_lag11,y_lag12
326,2019-04-01,0.0602,0.0366,0.0309,0.1876,-0.0560
327,2019-05-01,0.1122,0.1317,-0.1348,0.0309,0.1876
328,2019-06-01,-0.0197,0.0602,0.1258,-0.1348,0.0309
329,2019-07-01,-0.0544,0.1122,-0.1334,0.1258,-0.1348
330,2019-08-01,0.0514,-0.0197,0.1332,-0.1334,0.1258


In [25]:
y_pct_train = dataset_pct_train['sales change round %']
x_pct_train = pd.DataFrame(dataset_pct_train, columns = ['y_lag2','y_lag10', 'y_lag11', 'y_lag12'])
y_pct_test = dataset_pct_test['sales change round %']
x_pct_test = pd.DataFrame(dataset_pct_test, columns = ['y_lag2','y_lag10', 'y_lag11', 'y_lag12'])

x_pct_test, y_pct_test

(     y_lag2  y_lag10  y_lag11  y_lag12
 326  0.0366   0.0309   0.1876  -0.0560
 327  0.1317  -0.1348   0.0309   0.1876
 328  0.0602   0.1258  -0.1348   0.0309
 329  0.1122  -0.1334   0.1258  -0.1348
 330 -0.0197   0.1332  -0.1334   0.1258
 ..      ...      ...      ...      ...
 388 -0.0090   0.1844  -0.1891   0.0714
 389  0.1189  -0.0771   0.1844  -0.1891
 390 -0.0176   0.0199  -0.0771   0.1844
 391 -0.0573   0.0799   0.0199  -0.0771
 392  0.0597   0.0107   0.0799   0.0199
 
 [67 rows x 4 columns],
 326    0.0602
 327    0.1122
 328   -0.0197
 329   -0.0544
 330    0.0514
         ...  
 388   -0.0176
 389   -0.0573
 390    0.0597
 391   -0.1072
 392    0.1053
 Name: sales change round %, Length: 67, dtype: float64)

In [ ]:
random_forest = RandomForestRegressor(n_estimators = 100, criterion ='squared_error', max_depth = 100, bootstrap = True)
random_forest.fit(x_pct_train, y_pct_train)
y_pct_predictions = random_forest.predict(x_pct_test)

In [ ]:
mse = 0

y_pct_test = np.array(y_pct_test)
y_pct_predictions = np.array(y_pct_predictions)

for i in range(len(y_pct_test)):
    mse += (y_pct_test[i] - y_pct_predictions[i]) ** 2

mse_value_pct_RF = mse / len(y_pct_test)
mse_value_pct_RF

It can be noticed that the Mean Squared Error value is quite low, indicating the significant performance of the model.

### Part 5. ETS Forecasting

In this part 2 types of ETS forecasting models were chosen for consideration - AutoETS for the absolute values of the sales and its percentage change month to month. The AutoETS model was chosen due to being rather universal.

**1) AutoETS for absolute values**

In [ ]:
dataset_abs = pd.DataFrame(data, columns = ['date','sales'])
dataset_abs_train, dataset_abs_test = train_test_split(dataset_abs, test_size = 0.2, random_state = None, shuffle = False)

In [ ]:
y_abs_train = dataset_abs_train['sales']
y_abs_test = dataset_abs_test['sales']

plot_series(y_abs_train, y_abs_test, labels=['sales fact', 'sales to be predicted'])

In [62]:
forecaster = AutoETS(auto = True, sp = 12, n_jobs = -1)

forecaster.fit(dataset_abs_train['sales'])

forecasting_length = len(dataset_abs_test['date'])
forecast_horizon = np.arange(1, forecasting_length + 1)

y_abs_fact = dataset_abs_train['sales']
y_abs_predict = forecaster.predict(forecast_horizon)

plot_series(y_abs_fact, y_abs_predict, labels = ['y_abs_fact', 'y_abs_predict'])

**1) AutoETS for percentage change**

In [ ]:
dataset_pct = pd.DataFrame(data, columns = ['date','sales change round %'])
dataset_pct_train, dataset_pct_test = train_test_split(dataset_pct, test_size = 0.2, random_state = None, shuffle = False)

In [ ]:
y_pct_train = dataset_pct_train['sales change round %']
y_pct_test = dataset_pct_test['sales change round %']

plot_series(y_pct_train, y_pct_test, labels=["sales change fact", "sales change to be predicted"])

In [ ]:
forecaster = AutoETS(auto = True, sp = 12, n_jobs = -1)

forecaster.fit(dataset_pct_train['sales change round %'])

forecasting_length = len(dataset_pct_test['date'])
forecast_horizon = np.arange(1, forecasting_length + 1)

y_pct_fact = dataset_pct_train['sales change round %']
y_pct_predict = forecaster.predict(forecast_horizon)

plot_series(y_pct_fact, y_pct_predict, labels = ['y_pct_fact', 'y_pct_predict'])

I have decided to evaluate the performance of the model based on the Mean Squared Error of the result.

In [ ]:
mse = 0

y_pct_test = np.array(y_pct_test)
y_pct_predict = np.array(y_pct_predict)

for i in range(len(y_test)):
    mse += (y_pct_test[i] - y_pct_predict[i]) ** 2

mse_value_pct_ETS = mse / len(y_pct_test)
mse_value_pct_ETS

### Part 6. Surprise part - OLS (not so surprising though)

As a surprise part it was decided to construct AR models for both the absolute sales value and its percentage change month to month.

In [28]:
from statsmodels.regression.linear_model import OLS

In [29]:
data = pd.read_excel('US_alcohol_sales.xlsx')

**For the absolute values:**

In [30]:
dataset_abs = pd.DataFrame(data, columns = ['date','sales'])
dataset_abs_train, dataset_abs_test = train_test_split(dataset_abs, test_size = 0.2, random_state = None, shuffle = False)
dataset_abs_test

,date,sales
314,2018-04-01,11505
315,2018-05-01,13663
316,2018-06-01,14085
317,2018-07-01,12186
318,2018-08-01,13719
...,...,...
388,2024-06-01,16552
389,2024-07-01,15603
390,2024-08-01,16534
391,2024-09-01,14761


Firstly, we should indicate the significant regressors based on the analysis of the training dataset. Then, these regressors will be applied to the test set to make prediction of the dependent variable.

In [33]:
dataset_abs_train['y_lag1'] = dataset_abs_train['sales'].shift(1)
dataset_abs_train['y_lag2'] = dataset_abs_train['sales'].shift(2)
dataset_abs_train['y_lag3'] = dataset_abs_train['sales'].shift(3)
dataset_abs_train['y_lag4'] = dataset_abs_train['sales'].shift(4)
dataset_abs_train['y_lag5'] = dataset_abs_train['sales'].shift(5)
dataset_abs_train['y_lag6'] = dataset_abs_train['sales'].shift(6)
dataset_abs_train['y_lag7'] = dataset_abs_train['sales'].shift(7)
dataset_abs_train['y_lag8'] = dataset_abs_train['sales'].shift(8)
dataset_abs_train['y_lag9'] = dataset_abs_train['sales'].shift(9)
dataset_abs_train['y_lag10'] = dataset_abs_train['sales'].shift(10)
dataset_abs_train['y_lag11'] = dataset_abs_train['sales'].shift(11)
dataset_abs_train['y_lag12'] = dataset_abs_train['sales'].shift(12)

dataset_abs_train = dataset_abs_train.dropna()

dataset_abs_train.head()

,date,sales,y_lag1,y_lag2,y_lag3,y_lag4,y_lag5,y_lag6,y_lag7,y_lag8,y_lag9,y_lag10,y_lag11,y_lag12
12,1993-02-01,3261,3031.0,4936.0,4240.0,4259.0,4126.0,4137.0,4466.0,4529.0,4221.0,4564.0,4002.0,3458.0
13,1993-03-01,4160,3261.0,3031.0,4936.0,4240.0,4259.0,4126.0,4137.0,4466.0,4529.0,4221.0,4564.0,4002.0
14,1993-04-01,4377,4160.0,3261.0,3031.0,4936.0,4240.0,4259.0,4126.0,4137.0,4466.0,4529.0,4221.0,4564.0
15,1993-05-01,4307,4377.0,4160.0,3261.0,3031.0,4936.0,4240.0,4259.0,4126.0,4137.0,4466.0,4529.0,4221.0
16,1993-06-01,4696,4307.0,4377.0,4160.0,3261.0,3031.0,4936.0,4240.0,4259.0,4126.0,4137.0,4466.0,4529.0


In [34]:
x = dataset_abs_train[['y_lag1', 'y_lag2', 'y_lag3', 'y_lag4', 'y_lag5', 'y_lag6', 'y_lag7', 'y_lag8', 'y_lag9', 'y_lag10', 'y_lag11', 'y_lag12']]
y = dataset_abs_train['sales']

model = sm.OLS(y, x)
results = model.fit()

print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                  sales   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          1.164e+04
Date:                Fri, 03 Jan 2025   Prob (F-statistic):                        0.00
Time:                        18:17:15   Log-Likelihood:                         -2219.7
No. Observations:                 302   AIC:                                      4463.
Df Residuals:                     290   BIC:                                      4508.
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

After a number of trials it has been noticed that the most significant lag variables are the ones with t = t - 2, t - 10, t - 11, t - 12. The results of such AR(12) regression are displayed below.

In [37]:
x_abs_train = dataset_abs_train[['y_lag2', 'y_lag10', 'y_lag11', 'y_lag12']]
y_abs_train = dataset_abs_train['sales']

model = sm.OLS(y_abs_train, x_abs_train)
results = model.fit()

print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                  sales   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          3.457e+04
Date:                Fri, 03 Jan 2025   Prob (F-statistic):                        0.00
Time:                        18:43:21   Log-Likelihood:                         -2225.4
No. Observations:                 302   AIC:                                      4459.
Df Residuals:                     298   BIC:                                      4474.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Introducing lags into the test dataset:

In [31]:
dataset_abs_test['y_lag1'] = dataset_abs_test['sales'].shift(1)
dataset_abs_test['y_lag2'] = dataset_abs_test['sales'].shift(2)
dataset_abs_test['y_lag3'] = dataset_abs_test['sales'].shift(3)
dataset_abs_test['y_lag4'] = dataset_abs_test['sales'].shift(4)
dataset_abs_test['y_lag5'] = dataset_abs_test['sales'].shift(5)
dataset_abs_test['y_lag6'] = dataset_abs_test['sales'].shift(6)
dataset_abs_test['y_lag7'] = dataset_abs_test['sales'].shift(7)
dataset_abs_test['y_lag8'] = dataset_abs_test['sales'].shift(8)
dataset_abs_test['y_lag9'] = dataset_abs_test['sales'].shift(9)
dataset_abs_test['y_lag10'] = dataset_abs_test['sales'].shift(10)
dataset_abs_test['y_lag11'] = dataset_abs_test['sales'].shift(11)
dataset_abs_test['y_lag12'] = dataset_abs_test['sales'].shift(12)

dataset_abs_test = dataset_abs_test.dropna()

dataset_abs_test.head()

,date,sales,y_lag1,y_lag2,y_lag3,y_lag4,y_lag5,y_lag6,y_lag7,y_lag8,y_lag9,y_lag10,y_lag11,y_lag12
326,2019-04-01,12918,12184.0,10766.0,10386.0,15087.0,13724.0,13473.0,11889.0,13719.0,12186.0,14085.0,13663.0,11505.0
327,2019-05-01,14368,12918.0,12184.0,10766.0,10386.0,15087.0,13724.0,13473.0,11889.0,13719.0,12186.0,14085.0,13663.0
328,2019-06-01,14085,14368.0,12918.0,12184.0,10766.0,10386.0,15087.0,13724.0,13473.0,11889.0,13719.0,12186.0,14085.0
329,2019-07-01,13319,14085.0,14368.0,12918.0,12184.0,10766.0,10386.0,15087.0,13724.0,13473.0,11889.0,13719.0,12186.0
330,2019-08-01,14003,13319.0,14085.0,14368.0,12918.0,12184.0,10766.0,10386.0,15087.0,13724.0,13473.0,11889.0,13719.0


In [46]:
x_abs_test = dataset_abs_test[['y_lag2', 'y_lag10', 'y_lag11', 'y_lag12']]
y_abs_predict = results.predict(x_abs_test)

y_abs_predict

,0
326,11926.510153
327,14237.743874
328,14423.655482
329,12926.884033
330,14127.008082
...,...
388,17888.711165
389,15098.204593
390,17479.824847
391,16119.559788


Comparison of the actual and predicted values can be done via considering Mean Squared Error.

In [47]:
mse = 0

y_abs_test = np.array(y_abs_test)
y_abs_predict = np.array(y_abs_predict)

for i in range(len(y_abs_test)):
    mse += (y_abs_test[i] - y_abs_predict[i]) ** 2

mse_value_abs_OLS = mse / len(y_abs_test)
mse_value_abs_OLS

895413.0580980333

**For the percentage change:**

In [48]:
dataset_pct = pd.DataFrame(data, columns = ['date','sales change round %'])
dataset_pct_train, dataset_pct_test = train_test_split(dataset_pct, test_size = 0.2, random_state = None, shuffle = False)
dataset_pct_test

,date,sales change round %
314,2018-04-01,-0.0560
315,2018-05-01,0.1876
316,2018-06-01,0.0309
317,2018-07-01,-0.1348
318,2018-08-01,0.1258
...,...,...
388,2024-06-01,-0.0176
389,2024-07-01,-0.0573
390,2024-08-01,0.0597
391,2024-09-01,-0.1072


Firstly, let us calibrate the model and identify the significant regressors on the training set.

In [49]:
dataset_pct_train['y_lag1'] = dataset_pct_train['sales change round %'].shift(1)
dataset_pct_train['y_lag2'] = dataset_pct_train['sales change round %'].shift(2)
dataset_pct_train['y_lag3'] = dataset_pct_train['sales change round %'].shift(3)
dataset_pct_train['y_lag4'] = dataset_pct_train['sales change round %'].shift(4)
dataset_pct_train['y_lag5'] = dataset_pct_train['sales change round %'].shift(5)
dataset_pct_train['y_lag6'] = dataset_pct_train['sales change round %'].shift(6)
dataset_pct_train['y_lag7'] = dataset_pct_train['sales change round %'].shift(7)
dataset_pct_train['y_lag8'] = dataset_pct_train['sales change round %'].shift(8)
dataset_pct_train['y_lag9'] = dataset_pct_train['sales change round %'].shift(9)
dataset_pct_train['y_lag10'] = dataset_pct_train['sales change round %'].shift(10)
dataset_pct_train['y_lag11'] = dataset_pct_train['sales change round %'].shift(11)
dataset_pct_train['y_lag12'] = dataset_pct_train['sales change round %'].shift(12)

dataset_pct_train = dataset_pct_train.dropna()

dataset_pct_train.head()

,date,sales change round %,y_lag1,y_lag2,y_lag3,y_lag4,y_lag5,y_lag6,y_lag7,y_lag8,y_lag9,y_lag10,y_lag11,y_lag12
12,1993-02-01,0.0759,-0.3859,0.1642,-0.0045,0.0322,-0.0027,-0.0737,-0.0139,0.0730,-0.0752,0.1404,0.1573,-0.0003
13,1993-03-01,0.2757,0.0759,-0.3859,0.1642,-0.0045,0.0322,-0.0027,-0.0737,-0.0139,0.0730,-0.0752,0.1404,0.1573
14,1993-04-01,0.0522,0.2757,0.0759,-0.3859,0.1642,-0.0045,0.0322,-0.0027,-0.0737,-0.0139,0.0730,-0.0752,0.1404
15,1993-05-01,-0.0160,0.0522,0.2757,0.0759,-0.3859,0.1642,-0.0045,0.0322,-0.0027,-0.0737,-0.0139,0.0730,-0.0752
16,1993-06-01,0.0903,-0.0160,0.0522,0.2757,0.0759,-0.3859,0.1642,-0.0045,0.0322,-0.0027,-0.0737,-0.0139,0.0730


The chosen regressors include L(2), L(10), L(11) and L(12) (same ones as for the absolute values). As it can be seen from the table, they all are proven to be significant.

In [50]:
x_pct_train = dataset_pct_train[['y_lag2', 'y_lag10', 'y_lag11', 'y_lag12']]
y_pct_train = dataset_pct_train['sales change round %']

model = sm.OLS(y_pct_train, x_pct_train)
results = model.fit()

print(results.summary())

                                  OLS Regression Results                                 
Dep. Variable:     sales change round %   R-squared (uncentered):                   0.782
Model:                              OLS   Adj. R-squared (uncentered):              0.779
Method:                   Least Squares   F-statistic:                              267.0
Date:                  Fri, 03 Jan 2025   Prob (F-statistic):                    3.50e-97
Time:                          18:58:39   Log-Likelihood:                          382.97
No. Observations:                   302   AIC:                                     -757.9
Df Residuals:                       298   BIC:                                     -743.1
Df Model:                             4                                                  
Covariance Type:              nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------

Now, let us modify the test set and make predictions on it.

In [51]:
dataset_pct_test['y_lag1'] = dataset_pct_test['sales change round %'].shift(1)
dataset_pct_test['y_lag2'] = dataset_pct_test['sales change round %'].shift(2)
dataset_pct_test['y_lag3'] = dataset_pct_test['sales change round %'].shift(3)
dataset_pct_test['y_lag4'] = dataset_pct_test['sales change round %'].shift(4)
dataset_pct_test['y_lag5'] = dataset_pct_test['sales change round %'].shift(5)
dataset_pct_test['y_lag6'] = dataset_pct_test['sales change round %'].shift(6)
dataset_pct_test['y_lag7'] = dataset_pct_test['sales change round %'].shift(7)
dataset_pct_test['y_lag8'] = dataset_pct_test['sales change round %'].shift(8)
dataset_pct_test['y_lag9'] = dataset_pct_test['sales change round %'].shift(9)
dataset_pct_test['y_lag10'] = dataset_pct_test['sales change round %'].shift(10)
dataset_pct_test['y_lag11'] = dataset_pct_test['sales change round %'].shift(11)
dataset_pct_test['y_lag12'] = dataset_pct_test['sales change round %'].shift(12)

dataset_pct_test = dataset_pct_test.dropna()

dataset_pct_test.head()

,date,sales change round %,y_lag1,y_lag2,y_lag3,y_lag4,y_lag5,y_lag6,y_lag7,y_lag8,y_lag9,y_lag10,y_lag11,y_lag12
326,2019-04-01,0.0602,0.1317,0.0366,-0.3116,0.0993,0.0186,0.1332,-0.1334,0.1258,-0.1348,0.0309,0.1876,-0.0560
327,2019-05-01,0.1122,0.0602,0.1317,0.0366,-0.3116,0.0993,0.0186,0.1332,-0.1334,0.1258,-0.1348,0.0309,0.1876
328,2019-06-01,-0.0197,0.1122,0.0602,0.1317,0.0366,-0.3116,0.0993,0.0186,0.1332,-0.1334,0.1258,-0.1348,0.0309
329,2019-07-01,-0.0544,-0.0197,0.1122,0.0602,0.1317,0.0366,-0.3116,0.0993,0.0186,0.1332,-0.1334,0.1258,-0.1348
330,2019-08-01,0.0514,-0.0544,-0.0197,0.1122,0.0602,0.1317,0.0366,-0.3116,0.0993,0.0186,0.1332,-0.1334,0.1258


In [52]:
x_pct_test = dataset_pct_test[['y_lag2', 'y_lag10', 'y_lag11', 'y_lag12']]
y_pct_predict = results.predict(x_pct_test)

y_pct_predict

,0
326,-0.031343
327,0.194668
328,0.009439
329,-0.093997
330,0.091177
...,...
388,0.032220
389,-0.141315
390,0.159278
391,-0.078459


In [53]:
mse = 0

y_pct_test = np.array(y_pct_test)
y_pct_predict = np.array(y_pct_predict)

for i in range(len(y_pct_test)):
    mse += (y_pct_test[i] - y_pct_predict[i]) ** 2

mse_value_pct_OLS = mse / len(y_pct_test)
mse_value_pct_OLS

0.004253758491542758

### Part 7. Model Comparison

In order to compare the models' performance and suitability of each of the forecasting methods for the data, we can look at the methods' Mean Squared Errors. The idea is that the model with the least MSE is the best-forecasting one.

In [ ]:
print('MSE of the Random Forest Regression for the Percentage Change in Sales:', mse_value_pct_RF)
print('MSE of the ETS for the Percentage Change in Sales:', mse_value_pct_ETS)
print('MSE of the Autoregressive Model AR(12) for the Percentage Change in Sales:', mse_value_pct_OLS)

Based on the MSE results, ETS model appeared to be the best for prediction.

### Part 8. Fun.

Despite the fact that New Year has already passed, I've decided to attach a cute and kind short film about a hedgehog. Hope you enjoy!

https://youtu.be/okNRsJmBf_A?si=r4ajdUrlbvOtS5Lt